In [1]:
import os
import py3Dmol
%load_ext autoreload
%autoreload 2

os.chdir('..')
os.getcwd()

from models.epsnet import *
from utils.datasets import *
from utils.transforms import *
from utils.misc import *
from utils.chem import *

In [2]:
device = 'cuda:0'
model = DualEncoderEpsNetwork().to(device)
model.load_state_dict(torch.load("./param/cross_align.pt"))

<All keys matched successfully>

In [3]:
mol = Chem.SDMolSupplier(f"./mol/ligand_init.sdf", sanitize=False)[0]

In [4]:
batch1 = mol_to_graph_data_obj(mol)
batch2 = copy.deepcopy(batch1)

mean_pos = batch2['pos'].mean(0)
batch2['pos'] = batch2['pos'] - mean_pos
query_batch = Batch.from_data_list([batch1]).cuda()
reference_batch = Batch.from_data_list([batch2]).cuda()
query_batch.pos.normal_()

model.eval()

pos_gen, pos_gen_traj = model.DDIM_CFG_Sampling(
    query_batch=query_batch,
    reference_batch=reference_batch,
    n_steps=200
)
reference_batch['pos'] = reference_batch['pos'] + mean_pos.cuda()
pos_gen = pos_gen.cpu() + mean_pos.cpu()


sample: 0it [00:00, ?it/s]

/home/iljung0810/Lab/DiffAlign/models/epsnet/dualenc.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  merge_batch.graph_idx = torch.tensor(merge_batch.batch)
/home/iljung0810/Lab/DiffAlign/models/common.py:123: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /opt/conda/conda-bld/pytorch_1708025847094/work/torch/csrc/utils/tensor_new.cpp:618.)
  bgraph_adj = torch.sparse.LongTensor(


In [5]:
init_mol = set_rdmol_positions(mol, reference_batch[0]['pos'])
gen_mol = set_rdmol_positions(mol, pos_gen)

In [6]:
mblock = Chem.MolToMolBlock(init_mol)
view = py3Dmol.view(width=500, height=500)
view.addModel(mblock, 'mol')
mblock = Chem.MolToMolBlock(gen_mol)
view.addModel(mblock, 'mol')
view.setStyle({'model':0},{'stick':{'color': 'orange'}})
view.setStyle({'model':1},{'stick':{'color': 'green'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.